In [2]:
import os, sys
sys.path.append(os.path.dirname(os.getcwd()))

In [3]:
from utils import *

import pandas as pd

In [19]:
import yaml

with open("../data/acl2023/papers-export.yml", "r") as stream:
    results = yaml.safe_load(stream)

In [89]:
data_for_df = [{'id': item['id'], 'title': item['title']} for item in results]
df = pd.DataFrame(data_for_df)
df

,id,title
0,21,Pre-training Multi-party Dialogue Models with ...
1,23,Dual-Gated Fusion with Prefix-Tuning for Multi...
2,25,Aerial Vision-and-Dialog Navigation
3,34,Fixing MoE Over-Fitting on Low-Resource Langua...
4,35,A Survey of Deep Learning for Mathematical Rea...
...,...,...
1970,5858,Cross-Lingual Knowledge Distillation for Answe...
1971,5859,NarrowBERT: Accelerating Masked Language Model...
1972,5860,Enhancing Neural Topic Model with Multi-Level ...
1973,5867,Revisit Few-shot Intent Classification with PL...


In [90]:
df['area'] = None
df['interpretability'] = None

In [95]:
with open("../data/acl2023/program.yml", "r") as stream:
    program = yaml.safe_load(stream)

program

[{'end_time': datetime.datetime(2023, 7, 10, 12, 30),
  'start_time': datetime.datetime(2023, 7, 10, 11, 0),
  'subsessions': [{'end_time': datetime.datetime(2023, 7, 10, 12, 30),
    'location': 'Metropolitan East',
    'papers': [{'end_time': datetime.datetime(2023, 7, 10, 11, 15),
      'id': '2602',
      'start_time': datetime.datetime(2023, 7, 10, 11, 0)},
     {'end_time': datetime.datetime(2023, 7, 10, 11, 30),
      'id': '2874',
      'start_time': datetime.datetime(2023, 7, 10, 11, 15)},
     {'end_time': datetime.datetime(2023, 7, 10, 11, 45),
      'id': '3953',
      'start_time': datetime.datetime(2023, 7, 10, 11, 30)},
     {'end_time': datetime.datetime(2023, 7, 10, 12, 0),
      'id': '3680',
      'start_time': datetime.datetime(2023, 7, 10, 11, 45)},
     {'end_time': datetime.datetime(2023, 7, 10, 12, 15),
      'id': '2961',
      'start_time': datetime.datetime(2023, 7, 10, 12, 0)},
     {'end_time': datetime.datetime(2023, 7, 10, 12, 30),
      'id': '5836',
   

In [96]:
mapping_from_area_to_interpretability_boolean = {
    'Linguistic Diversity': False,
    'Information Extraction': False,
    'NLP Applications': False,
    'Findings Spotlights II': None,
    'Virtual Poster': None,
    'Findings Spotlights III': None,
    'Semantics: Sentence-level Semantics, Textual Inference, and Other Areas': False,
    'Summarization': False,
    'Posters': None,
    'Question Answering': False,
    'Semantics: Lexical': False,
    'Sentiment Analysis, Stylistic Analysis, and Argument Mining': True,
    'Phonology, Morphology, and Word Segmentation': False,
    'Generation': False,
    'Speech and Multimodality': False,
    'Multilingualism and Cross-Lingual NLP': False,
    'Dialogue and Interactive Systems': False,
    'Resources and Evaluation': False,
    'Theme: Reality Check': False, # not 100% sure about this one
    'Syntax: Tagging, Chunking, and Parsing': False,
    'Linguistic Theories, Cognitive Modeling, and Psycholinguistics': False,
    'Machine Learning for NLP': False,
    'Ethics and NLP': False,
    'Interpretability and Analysis of Models for NLP': True,
    'Machine Translation': False,
    'Discourse and Pragmatics': False,
    'Industry track: Interactive Systems, Speech': False,
    'Industry track: Model efficiency, Information Extraction': False,
    'Information Extraction / Generation': False,
    'Large Language Models': False,
    'Student Research Workshop': None,
    'Findings Spotlights I': None,
    'Language Grounding to Vision, Robotics, and Beyond': False,
    'Information Retrieval and Text Mining': False,
    'Computational Social Science and Cultural Analytics': False
}

In [97]:
for day in program:
    for subsession in day['subsessions']:
        area = subsession['title']
        for paper in subsession['papers']:
            if not paper['id'].isdigit():
                # these seem to be TACL papers
                # their ids are like T3967 and are not in the df
                continue
                
            paper_id = int(paper['id'])
            is_interpretability_research = mapping_from_area_to_interpretability_boolean[area]
            df.loc[df['id'] == paper_id, 'area'] = area
            df.loc[df['id'] == paper_id, 'interpretability'] = is_interpretability_research

In [98]:
df['interpretability'].value_counts(dropna=False)

interpretability
None     1791
False     169
True       15
Name: count, dtype: int64

In [105]:
df['doi'] = df['title'].apply(get_acl_anthology_doi)
df

,id,title,area,interpretability,doi
0,21,Pre-training Multi-party Dialogue Models with ...,Virtual Poster,None,10.18653/v1/2023.acl-long.533
1,23,Dual-Gated Fusion with Prefix-Tuning for Multi...,Virtual Poster,None,10.18653/v1/2023.findings-acl.572
2,25,Aerial Vision-and-Dialog Navigation,Virtual Poster,None,10.18653/v1/2023.findings-acl.190
3,34,Fixing MoE Over-Fitting on Low-Resource Langua...,Virtual Poster,None,10.18653/v1/2023.findings-acl.897
4,35,A Survey of Deep Learning for Mathematical Rea...,Posters,None,10.18653/v1/2023.acl-long.817
...,...,...,...,...,...
1970,5858,Cross-Lingual Knowledge Distillation for Answe...,None,None,10.18653/v1/2023.findings-acl.885
1971,5859,NarrowBERT: Accelerating Masked Language Model...,Posters,None,10.18653/v1/2023.acl-short.146
1972,5860,Enhancing Neural Topic Model with Multi-Level ...,None,None,10.18653/v1/2023.findings-acl.845
1973,5867,Revisit Few-shot Intent Classification with PL...,None,None,10.18653/v1/2023.findings-acl.706


In [106]:
df.to_csv('../data/acl2023/parsed_data.csv')